In [15]:
import pandas as pd

#Load train and test datasets
df = pd.read_json(r"C:\Users\victo\OneDrive\Desktop\amazon reviews\Cell_Phones_and_Accessories_5.json", lines = True)

#check for the first few rows of the dataset
print(df.head())

       reviewerID        asin      reviewerName helpful  \
0  A30TL5EWN6DFXT  120401325X         christina  [0, 0]   
1   ASY55RVNIL0UD  120401325X          emily l.  [0, 0]   
2  A2TMXE2AFO7ONB  120401325X             Erica  [0, 0]   
3   AWJ0WZQYMYFQ4  120401325X                JM  [4, 4]   
4   ATX7CZYFXI1KW  120401325X  patrice m rogoza  [2, 3]   

                                          reviewText  overall  \
0  They look good and stick good! I just don't li...        4   
1  These stickers work like the review says they ...        5   
2  These are awesome and make my phone look so st...        5   
3  Item arrived in great time and was in perfect ...        4   
4  awesome! stays on, and looks great. can be use...        5   

                                     summary  unixReviewTime   reviewTime  
0                                 Looks Good      1400630400  05 21, 2014  
1                      Really great product.      1389657600  01 14, 2014  
2                         

In [23]:
import re
import nltk
from nltk.corpus import stopwords

# Download stopwords
nltk.download('stopwords')

# Define the set of stopwords in English
stop_words = set(stopwords.words('english'))

# Function for text preprocessing
def preprocess_text(text):
    # Ensure the text is a string
    text = str(text)
    
    # 1. Lowercase the text
    text = text.lower()
    
    # 2. Remove punctuation and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # 3. Remove stopwords
    words = text.split()
    words = [word for word in words if word not in stop_words]
    
    # 4. Join words back into a single string
    return ' '.join(words)

# Apply the preprocessing function to the 'reviewText' column
df['cleaned_review'] = df['reviewText'].apply(preprocess_text)

# Check the cleaned text
print(df[['reviewText', 'cleaned_review']].head())


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\victo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                          reviewText  \
0  They look good and stick good! I just don't li...   
1  These stickers work like the review says they ...   
2  These are awesome and make my phone look so st...   
3  Item arrived in great time and was in perfect ...   
4  awesome! stays on, and looks great. can be use...   

                                      cleaned_review  
0  look good stick good dont like rounded shape a...  
1  stickers work like review says stick great sta...  
2  awesome make phone look stylish used one far a...  
3  item arrived great time perfect condition howe...  
4  awesome stays looks great used multiple apple ...  


In [27]:
# Function to map ratings to sentiment
def map_sentiment(rating):
    if rating > 3:
        return 1 #positive
    elif rating < 3:
        return 0 #negative
    else:
        return None #Neutral

# Apply the mapping function to the 'overall' column
df['sentiment'] = df['overall'].apply(map_sentiment)

# Drop rows with 'None' in the sentiment column (if you want to ignore neutral reviews)
df = df.dropna(subset=['sentiment'])

# Check the mapping
print(df[['overall', 'sentiment']].head())

   overall  sentiment
0        4        1.0
1        5        1.0
2        5        1.0
3        4        1.0
4        5        1.0


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF Vectorizer (limiting to top 5000 features)
tfidf = TfidfVectorizer(max_features=5000)

# Fit and transform the cleaned review text
X = tfidf.fit_transform(df['cleaned_review'])

# Target variable (sentiment)
y = df['sentiment']

# Check the shape of the feature matrix
print(X.shape)


(173000, 5000)


In [32]:
from sklearn.model_selection import train_test_split

# Split the data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the sizes of the training and testing sets
print(f"Train size: {X_train.shape}, Test size: {X_test.shape}")


Train size: (138400, 5000), Test size: (34600, 5000)


In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Train a Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

         0.0       0.83      0.59      0.69      4900
         1.0       0.94      0.98      0.96     29700

    accuracy                           0.92     34600
   macro avg       0.88      0.78      0.82     34600
weighted avg       0.92      0.92      0.92     34600

[[ 2890  2010]
 [  603 29097]]
